# Assignment 1
## Ferry Model | Group 14
## Imports

In [2]:
import salabim as sim, numpy as np, pandas as pd
import time, sys


## Time Data

In [25]:
CAR_NUMBERS = pd.read_csv("TimeTable.csv", sep=";")

## Variables

In [26]:
SAILING_TIME = sim.Triangular(10,18,13)
NUMBER_OF_CARS = sim.Triangular(70,80,75)
PAYMENT_TIME = sim.Triangular(1,4,2)
SIM_TIME = 60*24 # Time in minutes
LOADING_TIME = 1/6 # 10 seconds per car
UNLOADING_TIME = 5/60 # 5 seconds per time
WAITING_TIME_PREPAID = 0.5 # minutes
PERCENTAGE_PREPAID = 0.2 

REPLICATIONS = 10 # Number of experiment replications

## Components

In [40]:
class Car(sim.Component):
    def setup(self, cartype, paid, location):
        self.cartype = cartype # either tourist or employee
        self.paid = paid # true for prepaid, false for not prepaid
        self.location = location # either mainland or island

    def process(self):  
        # Wait
        if self.cartype == "employee":
            yield self.hold(1)
        else: 
            yield self.hold(5)
    
    def payforferry(self):
        #Only for not prepaid cars
        if not self.paid:



class CarGenerator(sim.Component):
    def setup(self, location, cartype):
        self.cartype = cartype # either tourist or employee
        self.location = location # either mainland or island

    def process(self):
        while True:
            #Get current time
            CurrentCarNumbers = CAR_NUMBERS[CAR_NUMBERS["time"]<= (env.now()/60)].tail(1)

            # Generate a car
            Car(cartype = self.cartype, paid = False, location= self.location)

            # Wait for the correct amount of time until creating the next car
            ## Get the number of cars
            number_cars = int(CurrentCarNumbers[str(self.cartype) + "_" + str(self.location)])

            ## Calculate the time span the cars arrive in 
            time_span = int(CAR_NUMBERS["time"].iloc[CurrentCarNumbers.index + 1]) - int(CAR_NUMBERS["time"].iloc[CurrentCarNumbers.index])

            ## Check if number of cars is greater than zero, then wait the correct amount of time, otherwise wait for the time interval
            if number_cars > 0:
                yield self.hold(60*time_span / number_cars)
            else:
                yield self.hold(60*time_span)



class Ferry(sim.Component):
    def setup(self, capacity, carsonferry, ferryrides):
        self.capacity = capacity # indicates how much space there is on the ferry
        self.carsonferry = carsonferry # indicates how many cars there are currently on the ferry 
        self.ferryrides = ferryrides # counts the amount of ferryrides done


    def process(self):  
        yield self.hold(5)

## Queues

In [42]:
mainland_line1, mainland_line2, mainland_line3 = sim.Queue('mainland_line1'), sim.Queue('mainland_line2'), sim.Queue('mainland_line3')
island_line1, island_line2, island_line3 = sim.Queue('island_line1'), sim.Queue('island_line2'), sim.Queue('island_line3')


line numbers prefixed by B refer to  <ipython-input-42-8b90d25e2ae8>
   B1                                  mainland_line1 create                
   B1                                  mainland_line2 create                
   B1                                  mainland_line3 create                
   B2                                  island_line1 create                  
   B2                                  island_line2 create                  
   B2                                  island_line3 create                  


### Creating and running the environment

In [41]:

# Create the Environment
env = sim.Environment(time_unit='minutes', trace= True)
env.modelname("Canadian Ferries Simulation")
CarGenerator(cartype="employee", location="island")
CarGenerator(cartype="employee", location="mainland")
CarGenerator(cartype="tourist", location="island")
CarGenerator(cartype="tourist", location="mainland")

env.run(duration=SIM_TIME)
print()

line#        time current component    action                               information
------ ---------- -------------------- -----------------------------------  ------------------------------------------------
                                       line numbers refers to               <ipython-input-41-7a961c8a7c18>
    2                                  default environment initialize       
    2                                  main create                          
    2       0.000 main                 current                              
    4                                  cargenerator.0 create                
                                       line numbers prefixed by A refer to  <ipython-input-40-e7caa9c44904>
    4                                  cargenerator.0 activate              scheduled for      0.000 @  A22  process=process
    5                                  cargenerator.1 create                
    5                                  cargenerator.1 activat